Пункт Г

Импорт библиотек

In [851]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import sympy as sp
from scipy.optimize import curve_fit

Составляем матрицу А

In [852]:
n = 100
A = np.full((n, n), 0, dtype = 'float64')

for i in range(n):
    A[0][i] = 1

for i in range(1, n - 1):
    A[i][i - 1] = 1
    A[i][i]     = 10
    A[i][i + 1] = 1

A[n - 1][-1] = 1
A[n - 1][-2] = 1

F = np.arange(n, 0, -1)

print(A, F, sep='\n')

[[ 1.  1.  1. ...  1.  1.  1.]
 [ 1. 10.  1. ...  0.  0.  0.]
 [ 0.  1. 10. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ... 10.  1.  0.]
 [ 0.  0.  0. ...  1. 10.  1.]
 [ 0.  0.  0. ...  0.  1.  1.]]
[100  99  98  97  96  95  94  93  92  91  90  89  88  87  86  85  84  83
  82  81  80  79  78  77  76  75  74  73  72  71  70  69  68  67  66  65
  64  63  62  61  60  59  58  57  56  55  54  53  52  51  50  49  48  47
  46  45  44  43  42  41  40  39  38  37  36  35  34  33  32  31  30  29
  28  27  26  25  24  23  22  21  20  19  18  17  16  15  14  13  12  11
  10   9   8   7   6   5   4   3   2   1]


Обратная матрица

In [853]:
A_inv = np.linalg.inv(A)

print(A_inv)

[[ 1.10102051e+00 -1.01020514e-01 -9.08153701e-02 ... -1.02167291e-01
   1.13519213e-02 -1.11237244e+00]
 [-1.11225659e-01  1.11225659e-01 -1.03092893e-03 ...  1.03209923e-02
  -1.14677693e-03  1.12372436e-01]
 [ 1.12360733e-02 -1.12360733e-02  1.01124659e-01 ... -1.04263195e-03
   1.15847995e-04 -1.13519213e-02]
 ...
 [-2.95101731e-97  2.95101731e-97 -2.65591558e-96 ...  1.02167291e-01
  -1.13519213e-02  1.13519213e-02]
 [ 3.27890812e-98 -3.27890812e-98  2.95101731e-97 ... -1.13519213e-02
   1.12372436e-01 -1.12372436e-01]
 [-3.27890812e-98  3.27890812e-98 -2.95101731e-97 ...  1.13519213e-02
  -1.12372436e-01  1.11237244e+00]]


Найдём число обусловленности и $\lambda_{min}$, $\lambda_{max}$ (двумя методами: степенным и расчётным). Для дальнейших расчетов используем евклидову норму (№3)

In [854]:
W, V = np.linalg.eig(A)

th_lambda_min = np.min(W)
th_lambda_max = np.max(W)

n_iter = 280

y_prev = A[0]
y      = A.dot(y_prev)

for i in range(n_iter):
    tmp    = y
    y      = A.dot(y_prev) 
    y_prev = tmp

lambda_max = np.linalg.norm(y, ord = 2) / np.linalg.norm(y_prev, ord = 2)

y_prev = A[0]
y      = A_inv.dot(y_prev)

for i in range(n_iter):
    tmp    = y
    y      = A_inv.dot(y_prev) 
    y_prev = tmp

lambda_min = 1 / (np.linalg.norm(y, ord = 2) / np.linalg.norm(y_prev, ord = 2))

print("Степенной метод для n_iter = 280:", lambda_min, lambda_max)

mu = np.linalg.norm(A, ord = 2) * np.linalg.norm(A_inv, ord = 2)

print("Теоретические значение:", th_lambda_min, th_lambda_max, "\nmu =", mu)

Степенной метод для n_iter = 280: 0.886833590902666 12.023285852567515
Теоретические значение: 0.8888888888888881 12.007560287214396 
mu = 30.768233436976274


Проверим главные миноры

In [855]:
flag = True 

for i in range(n):
    if (np.linalg.det(A[:i, :i]) == 0):  
        flag = False
    
print(flag)

True


LU-разложение

In [856]:
matrix_size = n

L = np.eye(matrix_size, dtype = 'float64')
U = np.full((matrix_size, matrix_size), 0, dtype = 'float64')

for i in range(matrix_size):
    for j in range(matrix_size):
        if i <= j:
            U[i, j] = A[i, j]
            for k in range(i):
                U[i, j] -= L[i, k] * U[k, j]
        else:
            L[i, j] = 1 / U[j, j] * A[i, j]
            for k in range(j):
                L[i, j] -= 1 / U[j, j] * L[i, k] * U[k, j]

print(L, U, sep = '\n')

[[1.         0.         0.         ... 0.         0.         0.        ]
 [1.         1.         0.         ... 0.         0.         0.        ]
 [0.         0.11111111 1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.10102051 1.         0.        ]
 [0.         0.         0.         ... 0.         0.10102051 1.        ]]
[[ 1.00000000e+00  1.00000000e+00  1.00000000e+00 ...  1.00000000e+00
   1.00000000e+00  1.00000000e+00]
 [ 0.00000000e+00  9.00000000e+00  0.00000000e+00 ... -1.00000000e+00
  -1.00000000e+00 -1.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+01 ...  1.11111111e-01
   1.11111111e-01  1.11111111e-01]
 ...
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  9.89897949e+00
   1.00000000e+00 -2.88841690e-96]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   9.89897949e+00  1.00000000e+00]
 [ 0.00000000e+00  0.00000

Ly = f \;
Ux = y

In [857]:
y = np.zeros(n)

for i in range(n):
    y[i] = F[i]
    for j in range(i):
        y[i] -= L[i, j] * y[j]
    y[i] /=  L[i, i]

x = np.zeros(n)

for i in range(n - 1, -1, -1):
    x[i] = y[i]
    for j in range(n - 1, i, -1):
        x[i] -= U[i, j] * x[j]
    x[i] /= U[i, i]

F_ = A.dot(x)
# print(F_)

print("Решение СЛАУ Ax = F:", x, sep = '\n')
print("Невязка прямого метода:", np.linalg.norm(F - A.dot(x), ord = 2))

Решение СЛАУ Ax = F:
[-3.45745028e+02  4.40191782e+01  4.55324589e+00  8.44836296e+00
  7.96312452e+00  7.92039185e+00  7.83295701e+00  7.75003802e+00
  7.66666283e+00  7.58333372e+00  7.49999996e+00  7.41666667e+00
  7.33333333e+00  7.25000000e+00  7.16666667e+00  7.08333333e+00
  7.00000000e+00  6.91666667e+00  6.83333333e+00  6.75000000e+00
  6.66666667e+00  6.58333333e+00  6.50000000e+00  6.41666667e+00
  6.33333333e+00  6.25000000e+00  6.16666667e+00  6.08333333e+00
  6.00000000e+00  5.91666667e+00  5.83333333e+00  5.75000000e+00
  5.66666667e+00  5.58333333e+00  5.50000000e+00  5.41666667e+00
  5.33333333e+00  5.25000000e+00  5.16666667e+00  5.08333333e+00
  5.00000000e+00  4.91666667e+00  4.83333333e+00  4.75000000e+00
  4.66666667e+00  4.58333333e+00  4.50000000e+00  4.41666667e+00
  4.33333333e+00  4.25000000e+00  4.16666667e+00  4.08333333e+00
  4.00000000e+00  3.91666667e+00  3.83333333e+00  3.75000000e+00
  3.66666667e+00  3.58333333e+00  3.50000000e+00  3.41666667e+00
  3.

Метод верхней релаксации

In [858]:
D = np.diag(np.diag(A))
L = np.tril(A) - D
U = np.triu(A) - D

u   = np.zeros(n)
u_0 = np.zeros(n)
omega = 1
eps = 1e-12

B   = (- np.linalg.inv(D + omega * L)).dot((omega - 1) * D + omega * U)
F_b = omega * (np.linalg.inv(D + omega * L)).dot(F)

k = 0

while (np.linalg.norm(F - A.dot(u), ord = 2) > eps):
    u = B.dot(u) + F_b
    k += 1

print(k)
print(u)
print("Невязка: ", np.linalg.norm(F - A.dot(u), ord = 2))
print("Критерий останова: невязка < eps = 1e-12")

16
[-3.45745028e+02  4.40191782e+01  4.55324589e+00  8.44836296e+00
  7.96312452e+00  7.92039185e+00  7.83295701e+00  7.75003802e+00
  7.66666283e+00  7.58333372e+00  7.49999996e+00  7.41666667e+00
  7.33333333e+00  7.25000000e+00  7.16666667e+00  7.08333333e+00
  7.00000000e+00  6.91666667e+00  6.83333333e+00  6.75000000e+00
  6.66666667e+00  6.58333333e+00  6.50000000e+00  6.41666667e+00
  6.33333333e+00  6.25000000e+00  6.16666667e+00  6.08333333e+00
  6.00000000e+00  5.91666667e+00  5.83333333e+00  5.75000000e+00
  5.66666667e+00  5.58333333e+00  5.50000000e+00  5.41666667e+00
  5.33333333e+00  5.25000000e+00  5.16666667e+00  5.08333333e+00
  5.00000000e+00  4.91666667e+00  4.83333333e+00  4.75000000e+00
  4.66666667e+00  4.58333333e+00  4.50000000e+00  4.41666667e+00
  4.33333333e+00  4.25000000e+00  4.16666667e+00  4.08333333e+00
  4.00000000e+00  3.91666667e+00  3.83333333e+00  3.75000000e+00
  3.66666667e+00  3.58333333e+00  3.50000000e+00  3.41666667e+00
  3.33333333e+00  3.25